In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Directory and file paths
train_dir = 'train_set/images'
val_dir = 'val_set/images'
train_ann_file = 'train_annotation.csv'
val_ann_file = 'val_annotation.csv'

# Parameters
input_shape = (224, 224, 3)  # Specify input image shape
batch_size = 24
num_emotions = 8

# Function to one-hot encode emotion labels
def one_hot_encode(number, num_classes=num_emotions):
    one_hot_vector = np.zeros(num_classes)
    one_hot_vector[number] = 1
    return one_hot_vector

# Training parameters
# epoch_steps = 30000 // batch_size
epoch_steps=30
val_steps = 30
epoch = 10

In [2]:
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import DepthwiseConv2D, GlobalAveragePooling2D, Activation, Reshape, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D
from tensorflow.keras import Model

In [3]:
from tensorflow.keras import backend as K

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [4]:
# Define CCC function
def ccc(y_true, y_pred):
    y_true_mean = tf.reduce_mean(y_true, axis=0)
    y_pred_mean = tf.reduce_mean(y_pred, axis=0)
    
    covariance = tf.reduce_mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    true_var = tf.reduce_mean((y_true - y_true_mean) ** 2)
    pred_var = tf.reduce_mean((y_pred - y_pred_mean) ** 2)
    
    ccc_value = 2 * covariance / (true_var + pred_var + tf.square(y_true_mean - y_pred_mean))
    
    return 1 - ccc_value

In [5]:
# Function to generate augmented data for minority class images
def data_generator(dir_path, ann_file, batch_size, input_shape, augment=False):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    minority_classes=[4,5,7]
    anno = pd.read_csv(ann_file)
    while True:
        batch_paths = np.random.choice(anno['filename'], size=batch_size)
        batch_input, batch_output_valence, batch_output_arousal, batch_output_emotion = [], [], [], []

        for input_path in batch_paths:
            img_path = os.path.join(dir_path, str(input_path)+'.jpg')
            img = load_img(img_path, target_size=input_shape)
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)  # Reshape to add batch dimension
            img_array = next(datagen.flow(img_array, batch_size=1))[0]
            row = anno[anno["filename"] == input_path]
            if row.empty:
                print(f"Warning: No annotation found for {input_path}")
                continue
            
            batch_input.append(img_array)
            batch_output_valence.append(row["Valance"].values[0])
            batch_output_arousal.append(row["Arousal"].values[0])
            batch_output_emotion.append(one_hot_encode(row["Expression"].values[0]))

        batch_input = np.array(batch_input)
        batch_output_valence = np.array(batch_output_valence)
        batch_output_arousal = np.array(batch_output_arousal)
        batch_output_emotion = np.array(batch_output_emotion)

        yield batch_input, {"valence": batch_output_valence, "arousal": batch_output_arousal, "emotion": batch_output_emotion}

In [6]:
#Downsampling majority class and upsampling minority class
def data_generator_down_up(dir_path, ann_file, batch_size, input_shape, augment=False):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    minority_classes = [4, 5, 7]
    ann_file = pd.read_csv(ann_file)
    majority_classes = ann_file[~ann_file["Expression"].isin(minority_classes)]
    minority_classes = ann_file[ann_file["Expression"].isin(minority_classes)]

    while True:
        batch_input, batch_output_valence, batch_output_arousal, batch_output_emotion = [], [], [], []

        majority_batch_paths = np.random.choice(majority_classes['filename'], size=batch_size // 2)
        minority_batch_paths = np.random.choice(minority_classes['filename'], size=batch_size // 2)
        # Determine the number of samples to take from minority and majority classes
        # minority_size = int(batch_size * 0.50)
        # majority_size = batch_size - minority_size
        
        # # Randomly select samples from the minority and majority annotations
        # minority_batch_paths = minority_classes.sample(n=minority_size, replace=True)['filename']
        # majority_batch_paths = majority_classes.sample(n=majority_size, replace=True)['filename']
        
        for input_path in np.concatenate((majority_batch_paths, minority_batch_paths)):
            img_path = os.path.join(dir_path, str(input_path)+'.jpg')
            img = load_img(img_path, target_size=input_shape)
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)  # Reshape to add batch dimension
            
            row = ann_file[ann_file["filename"] == input_path]
            if row.empty:
                print(f"Warning: No annotation found for {input_path}")
                continue
            
            # Check if the image belongs to a minority class
            emotion_class = row["Expression"].values[0]
            if augment and emotion_class in minority_classes:
                # Apply data augmentation
                for _ in range(3):  # Generate 3 augmented images
                    augmented_img = next(datagen.flow(img_array, batch_size=1))[0]
                    batch_input.append(augmented_img)  # Remove batch dimension
                    batch_output_valence.append(row["Valance"].values[0])
                    batch_output_arousal.append(row["Arousal"].values[0])
                    batch_output_emotion.append(one_hot_encode(row["Expression"].values[0]))
                    
            else:
                # No augmentation for majority classes
                batch_input.append(img_array[0])
                batch_output_valence.append(row["Valance"].values[0])
                batch_output_arousal.append(row["Arousal"].values[0])
                batch_output_emotion.append(one_hot_encode(row["Expression"].values[0]))

        batch_input = np.array(batch_input)
        batch_output_valence = np.array(batch_output_valence)
        batch_output_arousal = np.array(batch_output_arousal)
        batch_output_emotion = np.array(batch_output_emotion)

        yield batch_input, {"valence": batch_output_valence, "arousal": batch_output_arousal, "emotion": batch_output_emotion}

In [7]:
# Define the data generator function with augmentation and equal class distribution
def data_generator_equal(dir_path, ann_file, batch_size, input_shape, augment=False, num_classes=8, augment_count=1):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    anno = pd.read_csv(ann_file)
    class_counts = anno['Expression'].value_counts().to_dict()

    while True:
        # Initialize empty lists for the batch
        batch_input, batch_output_valence, batch_output_arousal, batch_output_emotion = [], [], [], []
        
        # Determine the number of samples per class
        samples_per_class = batch_size // num_classes
        
        for emotion_class in range(num_classes):
            class_anno = anno[anno["Expression"] == emotion_class]
            selected_paths = class_anno.sample(n=samples_per_class, replace=True)['filename']
            
            for input_path in selected_paths:
                img_path = os.path.join(dir_path, str(input_path)+'.jpg')
                try:
                    img = load_img(img_path, target_size=input_shape)
                    img_array = img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)  # Reshape to add batch dimension
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
                    continue
                
                row = class_anno[class_anno["filename"] == input_path]
                if row.empty:
                    print(f"Warning: No annotation found for {input_path}")
                    continue
                
                # Check if the image should be augmented
                if augment:
                    # Apply data augmentation
                    for _ in range(augment_count):  # Generate augment_count augmented images
                        augmented_img = next(datagen.flow(img_array, batch_size=1))[0]
                        batch_input.append(augmented_img)  # Remove batch dimension
                        batch_output_valence.append(row["Valance"].values[0])
                        batch_output_arousal.append(row["Arousal"].values[0])
                        batch_output_emotion.append(one_hot_encode(row["Expression"].values[0], num_classes=num_classes))
                else:
                    # No augmentation
                    batch_input.append(img_array[0])
                    batch_output_valence.append(row["Valance"].values[0])
                    batch_output_arousal.append(row["Arousal"].values[0])
                    batch_output_emotion.append(one_hot_encode(row["Expression"].values[0], num_classes=num_classes))

        # Convert lists to numpy arrays
        try:
            batch_input = np.array(batch_input)
            batch_output_valence = np.array(batch_output_valence)
            batch_output_arousal = np.array(batch_output_arousal)
            batch_output_emotion = np.array(batch_output_emotion)
        except Exception as e:
            print(f"Error converting batch to numpy arrays: {e}")
            continue
        yield batch_input, {"valence": batch_output_valence, "arousal": batch_output_arousal, "emotion": batch_output_emotion}

Model Definitioin: AlexNet

In [8]:

def swish(x):
    return x * tf.nn.sigmoid(x)

def SEBlock(inputs, reduction_ratio=4):
    channels = inputs.shape[-1]
    x = GlobalAveragePooling2D()(inputs)
    x = Dense(channels // reduction_ratio, activation='relu')(x)
    x = Dense(channels, activation='sigmoid')(x)
    x = Reshape((1, 1, channels))(x)
    return inputs * x

def MBConvBlock(inputs, expand_ratio, output_channels, stride):
    input_channels = inputs.shape[-1]
    expanded_channels = input_channels * expand_ratio
    
    x = Conv2D(expanded_channels, kernel_size=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation(swish)(x)
    
    x = DepthwiseConv2D(kernel_size=3, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(swish)(x)
    
    x = SEBlock(x)
    
    x = Conv2D(output_channels, kernel_size=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    if input_channels == output_channels and stride == 1:
        x = Add()([x, inputs])
    
    return x

def EfficientNet(input_shape, num_classes, alpha=1.0, beta=1.0):
    inputs = Input(shape=input_shape)
    x = inputs
    
    # Stem
    x = Conv2D(int(32 * alpha), kernel_size=3, strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(swish)(x)
    
    # Blocks
    block_settings = [
        {'num_repeat': 1, 'expand_ratio': 1, 'output_channels': 16, 'stride': 1},
        {'num_repeat': 2, 'expand_ratio': 6, 'output_channels': 24, 'stride': 2},
        {'num_repeat': 2, 'expand_ratio': 6, 'output_channels': 40, 'stride': 2},
        {'num_repeat': 3, 'expand_ratio': 6, 'output_channels': 80, 'stride': 2},
        {'num_repeat': 3, 'expand_ratio': 6, 'output_channels': 112, 'stride': 1},
        {'num_repeat': 4, 'expand_ratio': 6, 'output_channels': 192, 'stride': 2},
        {'num_repeat': 1, 'expand_ratio': 6, 'output_channels': 320, 'stride': 1}
    ]
    
    for settings in block_settings:
        num_repeat = settings['num_repeat']
        expand_ratio = settings['expand_ratio']
        output_channels = int(settings['output_channels'] * alpha)
        stride = settings['stride']
        
        for _ in range(num_repeat):
            x = MBConvBlock(x, expand_ratio, output_channels, stride)
            stride = 1
    
    # Head
    x = Conv2D(int(1280 * alpha), kernel_size=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(swish)(x)
    
    # Squeeze-and-Excitation
    x = SEBlock(x, reduction_ratio=beta)
    
    x = GlobalAveragePooling2D()(x)
    
    # Output for each task
    outputs = {
        'valence': Dense(1, activation='linear', name='valence')(x),
        'arousal': Dense(1, activation='linear', name='arousal')(x),
        'emotion': Dense(num_classes, activation='softmax', name='emotion')(x)
    }
    
    model = Model(inputs, outputs)
    return model


In [9]:
import gc
gc.collect()

0

In [ ]:
# Compile the model
modelOri = EfficientNet(input_shape, num_emotions)
modelOri.compile(optimizer=Adam(),
              loss={'valence': 'mse', 'arousal': 'mse', 'emotion': 'categorical_crossentropy'},
              metrics={'valence': ['mae','accuracy',ccc,rmse], 'arousal': ['mae','accuracy', ccc, rmse], 'emotion': ['accuracy']})

batch_size = 5  # Reduce the batch size
epoch_steps = 10  # Adjust according to your dataset
epoch = 5  # Number of epochs

# Training and validation data generators
train_genOri = data_generator(train_dir, train_ann_file, batch_size, input_shape)
val_genOri = data_generator(val_dir, val_ann_file, batch_size, input_shape)

# Callbacks
callbacks = [
    EarlyStopping(monitor='valence_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_modelOri.h5', save_best_only=True, monitor='valence_loss'),
    ReduceLROnPlateau(monitor='valence_loss', factor=0.2, patience=5, min_lr=1e-6)
]

# Train the model
historyOri = modelOri.fit(train_genOri, steps_per_epoch=epoch_steps, epochs=epoch, callbacks=callbacks, validation_data=val_genOri)

In [12]:
from numba import cuda

device = cuda.get_current_device()
device.reset()


In [ ]:
# Compile the model
modelOri = EfficientNet(input_shape, num_emotions)
modelOri.compile(optimizer=Adam(),
              loss={'valence': 'mse', 'arousal': 'mse', 'emotion': 'categorical_crossentropy'},
              metrics={'valence': ['mae','accuracy',ccc,rmse], 'arousal': ['mae','accuracy', ccc, rmse], 'emotion': ['accuracy']})
# input_shape = [224, 224]
# Training and validation data generators
train_genOri = data_generator(train_dir, train_ann_file, batch_size, input_shape)
val_genOri = data_generator(val_dir, val_ann_file, batch_size, input_shape)

# Callbacks
callbacks = [
    EarlyStopping(monitor='valence_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_modelOri.h5', save_best_only=True, monitor='valence_loss'),
    ReduceLROnPlateau(monitor='valence_loss', factor=0.2, patience=5, min_lr=1e-6)
]

# Train the model
historyOri = modelOri.fit(train_genOri, steps_per_epoch=epoch_steps, epochs=epoch, callbacks=callbacks)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[24,144,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/depthwise_conv2d_3/depthwise (defined at C:\Users\rijju\AppData\Local\Temp\ipykernel_4052\2767804707.py:19) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_13012]

Errors may have originated from an input operation.
Input Source operations connected to node model/depthwise_conv2d_3/depthwise:
 model/activation_7/mul (defined at C:\Users\rijju\AppData\Local\Temp\ipykernel_4052\821000116.py:2)

Function call stack:
train_function


In [ ]:
with open('plots/EfficientNethistoryOri.pkl', 'wb') as file:
    pickle.dump(historyOri.history, file)

In [ ]:
# Collect true labels and predictions
y_true_valence, y_pred_valence = [], []
y_true_arousal, y_pred_arousal = [], []
y_true_emotion, y_pred_emotion = [], []

resultallOri = modelOri.evaluate(val_genOri, steps=val_steps)

In [ ]:
# Convert the results to a DataFrame

df_results = pd.DataFrame([resultallOri], columns=modelOri.metrics_names)

# Save the DataFrame to a CSV file
df_results.to_csv("plots/csv_files_results/EfficientNetOri_results.csv", index=False)

In [ ]:
for _ in range(val_steps):
    x_val, y_val = next(val_genOri)
    y_pred = modelOri.predict(x_val)
    
    y_true_valence.extend(y_val['valence'])
    y_pred_valence.extend(y_pred[0].squeeze())  # Squeeze to match shape
    y_true_arousal.extend(y_val['arousal'])
    y_pred_arousal.extend(y_pred[1].squeeze())  # Squeeze to match shape
    y_true_emotion.extend(y_val['emotion'])
    y_pred_emotion.extend(y_pred[2])

# Convert lists to arrays
y_true_valence = np.array(y_true_valence)
y_pred_valence = np.array(y_pred_valence)
y_true_arousal = np.array(y_true_arousal)
y_pred_arousal = np.array(y_pred_arousal)
y_true_emotion = np.argmax(np.array(y_true_emotion), axis=1)
y_pred_emotion = np.argmax(np.array(y_pred_emotion), axis=1)

# Calculate Precision, Recall, and F1-Score for emotion
true_positives = np.sum((y_pred_emotion == y_true_emotion) & (y_true_emotion == 1))
predicted_positives = np.sum(y_pred_emotion == 1)
actual_positives = np.sum(y_true_emotion == 1)

precision = true_positives / predicted_positives if predicted_positives > 0 else 0
recall = true_positives / actual_positives if actual_positives > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall + np.finfo(float).eps)


print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
evaluation_metrics = {
    "Emotion precision": round(precision, 4),
    "Emotion recall": round(recall, 4),
    "Emotion F1-Score": round(f1_score, 4),
}

df_results = pd.DataFrame([evaluation_metrics])

# Save the DataFrame to a CSV file
df_results.to_csv("plots/csv_files_results/PRF_emotion_EfficientNetOri.csv", index=False)


In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(historyOri.history['valence_loss'], label='Valence Loss')
plt.plot(historyOri.history['arousal_loss'], label='Arousal Loss')
plt.plot(historyOri.history['emotion_loss'], label='Emotion Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(historyOri.history['valence_accuracy'], label='Valence Accuracy')
plt.plot(historyOri.history['arousal_accuracy'], label='Arousal Accuracy')
plt.plot(historyOri.history['emotion_accuracy'], label='Emotion Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig("plots/EfficientNetOritrainingloss_acc.pdf",bbox_inches='tight')
plt.show()

In [ ]:
modelCW = EfficientNet(input_shape, num_emotions)
# Calculate class weights
total_samples = 134415 + 74874 + 25459 + 24882 + 14090 + 6378 + 3803 + 3750
class_weights = {
    "valence": 1.0,
    "arousal": 1.0,
    "emotion": {
        0: total_samples / 134415,  # Happy
        1: total_samples / 74874,   # Neutral
        2: total_samples / 25459,   # Sad
        3: total_samples / 24882,   # Anger
        4: total_samples / 14090,   # Surprise
        5: total_samples / 6378,    # Fear
        6: total_samples / 3803,    # Disgust
        7: total_samples / 3750     # Contempt
    }
}

modelCW.compile(optimizer=Adam(),
              loss={'valence': 'mse', 'arousal': 'mse', 'emotion': 'categorical_crossentropy'},
              metrics={'valence': ['mae','accuracy',ccc, rmse], 'arousal': ['mae','accuracy', ccc, rmse], 'emotion': ['accuracy']},
              loss_weights=class_weights)  # Pass class weights to the loss_weights parameter

# Training and validation data generators
train_genCW = data_generator(train_dir, train_ann_file, batch_size, input_shape)
val_genCW = data_generator(val_dir, val_ann_file, batch_size, input_shape)


# Callbacks
callbacks = [
    EarlyStopping(monitor='valence_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_modelAlexNetWC.h5', save_best_only=True, monitor='valence_loss'),
    ReduceLROnPlateau(monitor='valence_loss', factor=0.2, patience=5, min_lr=1e-6)
]

# Train the model
historyCW = modelCW.fit(train_genCW, steps_per_epoch=epoch_steps, epochs=epoch, callbacks=callbacks)

In [ ]:
with open('plots/EfficientNethistoryCW.pkl', 'wb') as file:
    pickle.dump(historyCW.history, file)

In [ ]:
# Collect true labels and predictions
y_true_valence, y_pred_valence = [], []
y_true_arousal, y_pred_arousal = [], []
y_true_emotion, y_pred_emotion = [], []

resultallCW = modelCW.evaluate(val_genCW, steps=val_steps)

In [ ]:
# Convert the results to a DataFrame
df_results = pd.DataFrame([resultallCW], columns=modelCW.metrics_names)

# Save the DataFrame to a CSV file
df_results.to_csv("plots/csv_files_results/EfficientNetCW_results.csv", index=False)

In [ ]:
for _ in range(val_steps):
    x_val, y_val = next(val_genCW)
    y_pred = modelCW.predict(x_val)
    
    y_true_valence.extend(y_val['valence'])
    y_pred_valence.extend(y_pred[0].squeeze())  # Squeeze to match shape
    y_true_arousal.extend(y_val['arousal'])
    y_pred_arousal.extend(y_pred[1].squeeze())  # Squeeze to match shape
    y_true_emotion.extend(y_val['emotion'])
    y_pred_emotion.extend(y_pred[2])

# Convert lists to arrays
y_true_valence = np.array(y_true_valence)
y_pred_valence = np.array(y_pred_valence)
y_true_arousal = np.array(y_true_arousal)
y_pred_arousal = np.array(y_pred_arousal)
y_true_emotion = np.argmax(np.array(y_true_emotion), axis=1)
y_pred_emotion = np.argmax(np.array(y_pred_emotion), axis=1)

# Calculate Precision, Recall, and F1-Score for emotion
true_positives = np.sum((y_pred_emotion == y_true_emotion) & (y_true_emotion == 1))
predicted_positives = np.sum(y_pred_emotion == 1)
actual_positives = np.sum(y_true_emotion == 1)

precision = true_positives / predicted_positives if predicted_positives > 0 else 0
recall = true_positives / actual_positives if actual_positives > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall + np.finfo(float).eps)


print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
evaluation_metrics = {
    "Emotion precision": round(precision, 4),
    "Emotion recall": round(recall, 4),
    "Emotion F1-Score": round(f1_score, 4),
}

df_results = pd.DataFrame([evaluation_metrics])

# Save the DataFrame to a CSV file
df_results.to_csv("plots/csv_files_results/PRF_emotion_EfficientNetCW.csv", index=False)


In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(historyCW.history['valence_loss'], label='Valence Loss')
plt.plot(historyCW.history['arousal_loss'], label='Arousal Loss')
plt.plot(historyCW.history['emotion_loss'], label='Emotion Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(historyCW.history['valence_accuracy'], label='Valence Accuracy')
plt.plot(historyCW.history['arousal_accuracy'], label='Arousal Accuracy')
plt.plot(historyCW.history['emotion_accuracy'], label='Emotion Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig("plots/EfficientNetCWtrainingloss_acc.pdf",bbox_inches='tight')
plt.show()

In [ ]:
# Compile the model
modelUD = EfficientNet(input_shape, num_emotions)
modelUD.compile(optimizer=Adam(),
              loss={'valence': 'mse', 'arousal': 'mse', 'emotion': 'categorical_crossentropy'},
              metrics={'valence': ['mae','accuracy',ccc, rmse], 'arousal': ['mae','accuracy', ccc, rmse], 'emotion': ['accuracy']})

# Training and validation data generators
train_genUD = data_generator_down_up(train_dir, train_ann_file, batch_size, input_shape, augment=True)
val_genUD = data_generator(val_dir, val_ann_file, batch_size, input_shape)

# Callbacks
callbacks = [
    EarlyStopping(monitor='valence_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_modelOri.h5', save_best_only=True, monitor='valence_loss'),
    ReduceLROnPlateau(monitor='valence_loss', factor=0.2, patience=5, min_lr=1e-6)
]

# Train the model
historyUD = modelUD.fit(train_genUD, steps_per_epoch=epoch_steps, epochs=epoch, callbacks=callbacks)

In [ ]:
with open('plots/EfficientNethistoryUD.pkl', 'wb') as file:
    pickle.dump(historyUD.history, file)

In [ ]:
# Collect true labels and predictions
y_true_valence, y_pred_valence = [], []
y_true_arousal, y_pred_arousal = [], []
y_true_emotion, y_pred_emotion = [], []

resultallUD = modelUD.evaluate(val_genUD, steps=val_steps)

In [ ]:
# Convert the results to a DataFrame
df_results = pd.DataFrame([resultallUD], columns=modelUD.metrics_names)

# Save the DataFrame to a CSV file
df_results.to_csv("plots/csv_file_results/EfficientNetUD_results.csv", index=False)

In [ ]:
for _ in range(val_steps):
    x_val, y_val = next(val_genUD)
    y_pred = modelUD.predict(x_val)
    
    y_true_valence.extend(y_val['valence'])
    y_pred_valence.extend(y_pred[0].squeeze())  # Squeeze to match shape
    y_true_arousal.extend(y_val['arousal'])
    y_pred_arousal.extend(y_pred[1].squeeze())  # Squeeze to match shape
    y_true_emotion.extend(y_val['emotion'])
    y_pred_emotion.extend(y_pred[2])

# Convert lists to arrays
y_true_valence = np.array(y_true_valence)
y_pred_valence = np.array(y_pred_valence)
y_true_arousal = np.array(y_true_arousal)
y_pred_arousal = np.array(y_pred_arousal)
y_true_emotion = np.argmax(np.array(y_true_emotion), axis=1)
y_pred_emotion = np.argmax(np.array(y_pred_emotion), axis=1)

# Calculate Precision, Recall, and F1-Score for emotion
true_positives = np.sum((y_pred_emotion == y_true_emotion) & (y_true_emotion == 1))
predicted_positives = np.sum(y_pred_emotion == 1)
actual_positives = np.sum(y_true_emotion == 1)

precision = true_positives / predicted_positives if predicted_positives > 0 else 0
recall = true_positives / actual_positives if actual_positives > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall + np.finfo(float).eps)


print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
evaluation_metrics = {
    "Emotion precision": round(precision, 4),
    "Emotion recall": round(recall, 4),
    "Emotion F1-Score": round(f1_score, 4),
}

df_results = pd.DataFrame([evaluation_metrics])

# Save the DataFrame to a CSV file
df_results.to_csv("plots/csv_files_results/PRF_emotion_EfficientNetUD.csv", index=False)


In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(historyUD.history['valence_loss'], label='Valence Loss')
plt.plot(historyUD.history['arousal_loss'], label='Arousal Loss')
plt.plot(historyUD.history['emotion_loss'], label='Emotion Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(historyUD.history['valence_accuracy'], label='Valence Accuracy')
plt.plot(historyUD.history['arousal_accuracy'], label='Arousal Accuracy')
plt.plot(historyUD.history['emotion_accuracy'], label='Emotion Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig("plots/EfficientNetUDtrainingloss_acc.pdf",bbox_inches='tight')
plt.show()

In [ ]:
with open('plots/EfficientNethistoryOri.pkl', 'rb') as file:
    historyOri = pickle.load(file)
with open('plots/EfficientNethistoryCW.pkl', 'rb') as file:
    historyCW = pickle.load(file)
with open('plots/EfficientNethistoryUD.pkl', 'rb') as file:
    historyUD = pickle.load(file)

plt.figure(figsize=(10, 6))
plt.plot(historyOri['valence_loss'], label='EfficientNet (Original data)')
plt.plot(historyCW['valence_loss'], label='EfficientNet (Class weight)')
plt.plot(historyUD['valence_loss'], label='EfficientNet (Up/Down sampling)')

plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Valence Loss', fontsize=14)
plt.legend(fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Valence Loss During Training', fontsize=16)

plt.subplots_adjust(left=0.1, right=0.95, top=0.9, bottom=0.15)

plt.savefig("plots/EfficientNet_valenceLoss.pdf", bbox_inches='tight')
plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# emotion_mapping = {
#     0: 'Neutral',
#     1: 'Happy',
#     2: 'Sad',
#     3: 'Surprise',
#     4: 'Fear',
#     5: 'Disgust',
#     6: 'Anger',
#     7: 'Contempt'
# }
# # Plot confusion matrix for emotion prediction with mapped labels
# emotion_labels_mapped = [emotion_mapping[i] for i in range(num_emotions)]
# cm_emotion_mapped = confusion_matrix(y_true_emotion, y_pred_emotion)
# plt.figure(figsize=(10, 8))
# sns.heatmap(cm_emotion_mapped, annot=True, fmt='d', cmap='Blues', xticklabels=emotion_labels_mapped, yticklabels=emotion_labels_mapped)
# # plt.title('Emotion Prediction Confusion Matrix')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()